In [ ]:
import requests as http
import pandas as pd

import numpy as np

import matplotlib.pyplot as plt

import seaborn as sns

path = 'https://financialmodelingprep.com/api/v3/stock/list?apikey=60ea23d5071d16bec275682c1267d941'

def get_tickers(path):
    symbols_list = []
    
    data= http.get(path)
    data = data.json()
    data = data[:10]
    return data
    

def acquire():
    
    df = pd.DataFrame()
    symbol_data = get_tickers(path)
    
    symbol_list= []
    for i in symbol_data:
        symbol_list.append(i['symbol'])
    
    for i in symbol_list:
        prices  = http.get(f'https://financialmodelingprep.com/api/v3/historical-price-full/{i}?serietype=line&apikey=60ea23d5071d16bec275682c1267d941').json()
        prices = prices['historical']
        
        if len(prices) <=3000:

            continue
        prices=prices[:3001]
        prices_list = []
        for x in prices:
            
            
            prices_list.append(x['close'])
            
        df[i] = prices_list  
        
    dates = http.get('https://financialmodelingprep.com/api/v3/historical-price-full/AAPL?serietype=line&apikey=60ea23d5071d16bec275682c1267d941').json()
    dates = dates['historical']
    dates = dates[:3001]
    dates_list = []
    for i in dates:
        dates_list.append(i['date'])
        
        
    df['new index'] = dates_list
    df = df.set_index('new index')
    
    
    return df

def compute_returns(df):
    
    returns = df.pct_change()
    return returns

def return_risk_ratio(df, days=30):
    means = pd.DataFrame(df.head(days).mean())
    std = pd.DataFrame(df.head(days).std())
    ratios = pd.concat([means, std], axis = 1).reset_index()
    ratios.columns = ['Company', 'Mean', 'Std']
    ratios['Ratio'] = ratios['Mean'] / ratios['Std']
    return ratios

def corr_matrix(df, days=30):
    corr_matrix = df.head(days).corr()
    return corr_matrix

def barchart(df, x,y, length=8, width=14, title=''):
    df = df.sort_values(x, ascending = False)
    plt.figure(figsize = (width, length))
    chart = sns.barplot(data=df, x=x, y=y)
    plt.title(title + '\n', fontsize=16)
    return chart


def correlation_plot(corr, title=''):
    mask = np.zeros_like(corr, dtype=np.bool)
    mask[np.triu_indices_from(mask)] = True
    
    plt.subplots(figsize=(15,10))
    cmap = sns.diverging_palette(6,255, as_cmap=True)
    chart = sns.heatmap(corr, mask= mask, cmap=cmap, center = 0, linewidths = .5, annot=True, fmt='.2f')
    plt.title(title, fontsize = 16)
    return chart

def save_viz(chart, title):
    fig = chart.get_figure()
    fig.savefig(title+ '.png')
    
def analyze(returns, days=30):
    ratios = return_risk_ratio(returns, days=days)
    top10 = ratios.sort_values('Ratio', ascending = False).head(10)
   
    target_list = returns[list(top10['Company'])]
    correlation = corr_matrix(target_list)
    return top10, correlation


def report(top10, correlation):
    bar_plot = barchart(top10, 'Ratio', 'Company', title = 'Stock Return vs Risk Ratios' + str(day) + 'Days')
    save_viz(bar_plot, 'Return vs. Risk top 10' + str(day) + 'Days')
    corr_plot = correlation_plot(correlation, title='Stock Return Correlation - ' + str(day) + ' Days')
    save_viz(corr_plot, 'Correlation Plot - ' + str(day) + ' Days')

    
if __name__ == '__main__':
    data = acquire()
    
    
    num_days = [90,180,360]
    
    for day in num_days:
        top10, correlation = analyze(data, days=day)
        report(top10, correlation)